#Installation of libraries

In [1]:
!pip install transformers


In [ ]:
!pip install huggingface-cli
!huggingface-cli login


ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

# Import & demo Implementations

In [ ]:
import requests
import time

def call_huggingface_endpoint(prompt, api_url, api_token, max_new_tokens, do_sample, temperature, top_p, max_length=512, retries=1, backoff_factor=0.3):
    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json"
    }
    data = {
        "inputs": prompt,
        "parameters": {
            "max_length": max_length,
            "max_new_tokens": max_new_tokens,
            "do_sample": do_sample,
            "temperature": temperature,
            "top_p": top_p,
        }
    }
    for attempt in range(retries):
        try:
            response = requests.post(api_url, headers=headers, json=data)
            response.raise_for_status()
            response_text = response.json()[0]["generated_text"]
            return response_text
        except requests.exceptions.RequestException as e:
            if attempt < retries - 1:
                sleep_time = backoff_factor * (2 ** attempt)
                print(f"Retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
            else:
                raise e


In [ ]:
def fetch_info(response, api_url, api_token):
    modelPromptForAppointment = f"""
    system
    Extract the following information from {response}: First Name, Last Name, Date Of Birth, Email Address, Phone Number, and Preferred Date Or Time|eot_id|>user
    {response}assistant
    """
    try:
        fields = ['FirstName', 'LastName', 'DateOfBirth', 'PhoneNumber', 'Email', 'Preferred date or time']
        result = call_huggingface_endpoint(modelPromptForAppointment, api_url, api_token, 256, False, 0.1, 0.9)
        result = result[len(modelPromptForAppointment):].strip()
        info = {}
        for line in result.split('\n'):
            if ':' in line:
                key, value = line.split(':', 1)
                if key.strip() in fields:
                    info[key.strip()] = f"{(value.strip()).replace('na','')}"
        print(info)
        return info
    except Exception as e:
        print(f"Error extracting information: {e}")
        return {}


### demo Chatbot

In [ ]:
# def chatbot():
#     api_url = 'Your Api url'
#     api_token = 'your Api token'

#     print("Chatbot is running. Type 'exit' to end the chat.")
#     while True:
#         user_input = input("You: ")
#         if user_input.lower() in ["quit", "exit", "bye"]:
#             print("Chat ended.")
#             break

#         response = fetch_info(user_input, api_url, api_token)
#         print(f"Chatbot: {response}")

# if __name__ == "__main__":
#     chatbot()


Chatbot is running. Type 'exit' to end the chat.
You: hello
{}
Chatbot: {}
You: book appointment
{}
Chatbot: {}
You: my name is Ayushi Raj
{}
Chatbot: {}
You: exit
Chat ended.


#Final Code For chatbot

In [ ]:
import re
import requests
import time

# Define the Hugging Face API details
api_url = 'your api url'
hugging_face_api_token = 'your token'

def call_huggingface_endpoint(prompt, api_url, api_token, max_new_tokens, do_sample, temperature, top_p, max_length=512, retries=1, backoff_factor=0.3):
    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json"
    }
    data = {
        "inputs": prompt,
        "parameters": {
            "max_length": max_length,
            "max_new_tokens": max_new_tokens,
            "do_sample": do_sample,
            "temperature": temperature,
            "top_p": top_p,
        }
    }
    for attempt in range(retries):
        try:
            response = requests.post(api_url, headers=headers, json=data)
            response.raise_for_status()
            response_text = response.json()[0]["generated_text"]
            return response_text
        except requests.exceptions.RequestException as e:
            if attempt < retries - 1:
                sleep_time = backoff_factor * (2 ** attempt)
                time.sleep(sleep_time)
            else:
                raise e

def fetch_info(response):
    modelPromptForAppointment = f"""
    Extract the following information from the text: First Name, Last Name, Date Of Birth, Email, Phone Number, and Preferred date or time if available.
    User's input: {response}
    """
    try:
        result = call_huggingface_endpoint(modelPromptForAppointment, api_url, hugging_face_api_token, 256, False, 0.1, 0.9)
        data_dict = {}
        for line in result.split('\n'):
            if ':' in line:
                pattern = r"([a-zA-Z\s]+):\s*(.+)"
                matches = re.findall(pattern, line)
                if matches:
                    key, value = matches[0][0].strip(), matches[0][1].strip()
                    data_dict[key] = value.replace('(not provided)', '').replace('(Not provided)', '')

        # Ensure correct mapping of keys
        mapped_dict = {
            "First Name": data_dict.get("First Name", ""),
            "Last Name": data_dict.get("Last Name", ""),
            "Date Of Birth": data_dict.get("Date Of Birth", ""),
            "Email": data_dict.get("Email", ""),
            "Phone Number": data_dict.get("Phone Number", ""),
            "Preferred Date Or Time": data_dict.get("Preferred Date Or Time", ""),
        }
        return mapped_dict
    except Exception as e:
        print(f"Error extracting information: {e}")
        return {}

def check_missing_info(info):
    required_fields = ['First Name', 'Last Name', 'Date Of Birth', 'Phone Number', 'Email', 'Preferred Date Or Time']
    missing_fields = [field for field in required_fields if not info.get(field)]
    return missing_fields

def handle_booking_request():
    print("Please provide the following information to book an appointment:")
    print("First Name, Last Name, Date Of Birth, Phone Number, Email, Preferred Date and Time")

    user_input = input("You: ")
    extracted_info = fetch_info(user_input)
    missing_fields = check_missing_info(extracted_info)

    while missing_fields:
        for field in missing_fields:
            print(f"Please provide the {field}:")
            user_input = input("You: ")
            extracted_info[field] = user_input
        missing_fields = check_missing_info(extracted_info)

    print("All required fields are provided. We are DONE !!!")
    print("Your appointment has been booked with the following details:")
    for key, value in extracted_info.items():
        print(f"{key}: {value}")

if __name__ == "__main__":
    print("Chatbot is running. Type 'I want to book an appointment' to start the process or 'exit' to end the chat.")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit", "bye"]:
            print("Chat ended.")
            break
        elif "book an appointment" in user_input.lower():
            handle_booking_request()
        else:
            print("To book an appointment, please type: 'I want to book an appointment'")


Chatbot is running. Type 'I want to book an appointment' to start the process or 'exit' to end the chat.
You: book an appointment
Please provide the following information to book an appointment:
First Name, Last Name, Date Of Birth, Phone Number, Email, Preferred Date and Time
You: my name is ayushi raj. my date of birth is 8/01/2003, my contact number is 8235856364, email is abc@gmail.com, i want to book an appointment for 9/01/2023
Please provide the Preferred Date Or Time:
You: 9 august 2024 3 pm
All required fields are provided. We are DONE !!!
Your appointment has been booked with the following details:
First Name: ayushi
Last Name: raj
Date Of Birth: 01/08/2003
Email: abc@gmail.com
Phone Number: 8235856364
Preferred Date Or Time: 9 august 2024 3 pm
You: 3 pm
To book an appointment, please type: 'I want to book an appointment'
You: exit
Chat ended.
